In [2]:
import numpy as np
import pandas as pd
import csv
import re
import random

In [3]:
#크롤링한 네이버 뉴스제목 읽기
f = open('섹션별_뉴스제목_데이터.csv', 'r')
new = csv.reader(f)
tmp_data = []
f.seek(0)

#리스트에 읽어온 데이터 삽입
for i in new:
    tmp_data.append(i)
    
f.close()

#리스트를 넘파이 배열로 변환
random.shuffle(tmp_data)    
data = np.array(tmp_data)

#7:3 비율로 데이터셋 분리
train = data[:11200]
test = data[11200:]

#트레인과 테스트 데이터 열에 이름을 붙임
train_txt = train[:,0]
train_label = train[:,1]
test_txt = test[:,0]
test_label = test[:,1]

In [5]:
import transformers
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

def text_processing(text, Max_len):
  input_ids = []
  mask_ids = []
  token_type_ids = []
  for line in text:
    # [cls], [sep] 추가 Max_len 길이 맞추기  
    encoded_line = tokenizer.encode(line, add_special_tokens = True, max_length=Max_len, truncation=True) 
    input = encoded_line + [0]*(Max_len-len(encoded_line))
    token_type =  [0]*len(input)
    mask = [1]*len(encoded_line)+[0]*(Max_len-len(encoded_line))    
    input_ids.append(torch.tensor(input))
    token_type_ids.append(torch.tensor(token_type))
    mask_ids.append(torch.tensor(mask))

    return input_ids, token_type_ids, mask_ids
 